In [17]:
# Import libraries
import os
from tqdm import tqdm
import pandas as pd

# Redirecting to the directory
# os.chdir(os.path.join('.\ECON4999X'))
CLEANED_DATA_DIR = os.path.normpath(r'.\Cleaned_Data')
DA_DATA_DIR = os.path.normpath(r'.\da_working_files')
print(os.listdir(CLEANED_DATA_DIR))

['df_exchange_info_master.xlsx', 'df_exchange_mapping.csv', 'df_ghg_ts_long_form.csv', 'df_ghg_ts_long_form_selected.csv', 'df_ghg_ts_master.csv', 'df_info_master.csv', 'df_power_ts_long_form.csv', 'df_power_ts_long_form_selected.csv', 'df_power_ts_master.csv', 'df_selected_comps.csv', 'hardcode_data.xlsx', '~$hardcode_data.xlsx']


### Does higher carbon price reduce firms’ carbon emissions?
#### Emission Analysis

In [14]:
# Reading dataframe
df_ghg = pd.read_csv(os.path.join(CLEANED_DATA_DIR, 'df_ghg_ts_long_form_selected.csv'))
df_ghg.head()

# We want to make a dataframe that only contains entity with long enough history
entity_history_requirement = 15
df_ghg_long_hist_comps = df_ghg.groupby('entity_id').filter(lambda x: len(x['year']) >= entity_history_requirement)

In [20]:
# Create a column called the total carbon emission
df_ghg_long_hist_comps['total_carbon_emission'] = df_ghg_long_hist_comps[['absolute_ghg_scope_1', 'absolute_ghg_scope_2']].sum(axis=1)

In [24]:
file_name = 'df_ghg_ts_long_form_selected_long_hist.csv'
df_ghg_long_hist_comps.to_csv(os.path.join(DA_DATA_DIR, file_name), index=False)

### How firms reduce the carbon emissions, by developing in-house green technology or cutting their output?